# AI 語音降噪 Colab
這個 Colab 筆記本會使用 [SpeechBrain](https://github.com/speechbrain/speechbrain) 的 MetricGAN+ 模型，使用 GPU 進行語音降噪。

載入 Google Drive 中的影音或聲音檔案，並將結果產生為聲音檔 (.wav)。

In [ ]:
#@title 1. 安裝相依套件並掛載雲端硬碟
!pip install -q speechbrain torchaudio ffmpeg-python
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 2. 設定來源檔案或資料夾與輸出路徑
input_path = '/content/drive/MyDrive/input.mp4'  #@param {type:'string'}
output_path = '/content/drive/MyDrive/output'  #@param {type:'string'}
use_gpu = True  #@param {type:'boolean'}


In [ ]:
#@title 3. 執行降噪
import os, glob, subprocess, torch, torchaudio
from speechbrain.pretrained.interfaces import SpectralMaskEnhancement

video_exts = ['.mp4','.mov','.avi','.mkv','.flv','.webm']
audio_exts = ['.wav','.mp3','.flac','.ogg','.m4a']

def extract_audio(src, dst):
    subprocess.run(['ffmpeg','-y','-i', src, '-vn','-ac','1', dst], check=True)

def denoise_file(src, dst, enhancer):
    tmp_wav = '/content/tmp_input.wav'
    ext = os.path.splitext(src)[1].lower()
    if ext in video_exts:
        extract_audio(src, tmp_wav)
        processed_input = tmp_wav
    else:
        processed_input = src
    enhanced = enhancer.enhance_file(processed_input)
    torchaudio.save(dst, enhanced.squeeze(0), 16000)
    if processed_input == tmp_wav and os.path.exists(tmp_wav):
        os.remove(tmp_wav)

device = 'cuda' if (use_gpu and torch.cuda.is_available()) else 'cpu'
enhancer = SpectralMaskEnhancement.from_hparams(
    source='speechbrain/metricgan-plus-voicebank',
    savedir='pretrained_models/metricgan-plus-voicebank',
    run_opts={'device': device}
)

if os.path.isdir(input_path):
    os.makedirs(output_path, exist_ok=True)
    candidates = glob.glob(os.path.join(input_path, '*'))
    files = [f for f in candidates if os.path.splitext(f)[1].lower() in video_exts + audio_exts]
    for f in sorted(files):
        out_file = os.path.join(output_path, os.path.splitext(os.path.basename(f))[0] + '.wav')
        if os.path.exists(out_file):
            print('跳過已存在的輸出檔案', out_file)
            continue
        denoise_file(f, out_file, enhancer)
        print('已處理', f)
else:
    if os.path.exists(output_path):
        print('輸出檔案已存在，跳過', output_path)
    else:
        denoise_file(input_path, output_path, enhancer)
        print('已處理', input_path)

print('處理完成')
